📦 1. Imports and Initial Configuration

In [ ]:
import pandas as pd
import numpy as np
import sys
import traceback

🧾 2. Input Data and Evaluation Parameters

In [ ]:
evaluations = np.array([
    [5, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 3],
    [5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5],
    ...
])

CRITERIA_NAMES = [
    "Reduces Congestion",
    "Efficiency of Land Transport",
    "Mitigates Accident Risk",
    "Reduces Road Wear",
    "Economies of Scale",
    "Reduces Fuel Consumption",
    "Reduces Cost per Ton",
    "Reduces Accident Records",
    "Reduces Cargo Damage",
    "Reduces Cargo Theft",
    "Reduces CO2 Emissions",
    "Minimizes Brazil Cost"
]

SAATY_SCALE_MAP = {
    0: 1,
    1: 3,
    2: 5,
    3: 7,
    4: 9,
    -1: 1/3,
    -2: 1/5,
    -3: 1/7,
    -4: 1/9
}

MAX_RATING = 5
MIN_RATING = 1
MAX_DIFF = MAX_RATING - MIN_RATING

RI_TABLE = {
    1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32,
    8: 1.41, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.48, 13: 1.56, 14: 1.57, 15: 1.59
}

🧮 3. Function: Map Difference to Saaty Scale

In [ ]:
def map_difference_to_saaty(diff):
    """Maps the difference between two scores to the Saaty scale."""
    clamped_diff = np.clip(diff, -MAX_DIFF, MAX_DIFF)
    rounded_diff = round(clamped_diff)
    return SAATY_SCALE_MAP.get(rounded_diff, 1)

🧩 4. Function: Convert Ratings to Pairwise Comparison Matrix

In [ ]:
def ratings_to_pcm(ratings_row):
    """Converts a row of ratings into a pairwise comparison matrix (PCM)."""
    n = len(ratings_row)
    pcm = np.ones((n, n))
    if np.isnan(ratings_row).any() or np.isinf(ratings_row).any():
        print("  Warning: Rating row contains NaN or Inf. Skipping respondent.", file=sys.stderr)
        return None
    for i in range(n):
        for j in range(i + 1, n):
            diff = ratings_row[i] - ratings_row[j]
            saaty_value = map_difference_to_saaty(diff)
            pcm[i, j] = saaty_value
            pcm[j, i] = 1.0 / saaty_value if saaty_value != 0 else np.inf
    if np.isnan(pcm).any() or np.isinf(pcm).any() or (pcm <= 0).any():
        print("  Warning: Invalid PCM (NaN/Inf/<=0). Skipping.", file=sys.stderr)
        return None
    return pcm